In this notebook we run the shock-acoustic-wave interaction problems using the CNS solver

In [ ]:
#r "BoSSSpad.dll"
#r "CNS.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
Init();


init workflow

In [ ]:
BoSSSshell.WorkflowMgm.Init("CNS_AcousticWave1D_ConvStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


In [ ]:
BoSSSshell.GetDefaultQueue()

In [ ]:
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path; 
dbPath

choose parameters (waveform, configuration, grid resolution...)

In [ ]:
double[] PerStartTimes = new double[] {10}; 
int[] Configs = new int[] {1}; //three interaction problems
int[] DGdegrees = new int[] {1,2,3};
int[] Grid_Res = new int[] {0,1,3,7};
double[] Amplitudes = new double[] {1e-5};
string[] WaveForms= new string[] {"1sinus"};
string[] LevelSetConfigs = new string[] {"true"};
double[] MachLs = new double[] {1.5};
double[] WLs = new double[] {0.8};

send the simulations to execution queue

In [ ]:
int savePeriod=1500;
double endTime=14.0;
double shockPos=1.5;
double wavePos=-0.4; 

bool isRestart=false;

foreach(double waveLength in WLs ){
foreach(double MachL in MachLs ){
foreach(double amp in Amplitudes ){
    double amp_neg=0.0;
    double amp_pos=0.0;
    
    foreach(double config in Configs) {
    foreach(string waveform in WaveForms) {
    foreach(double perStartTime in PerStartTimes) {
        if(config ==1){ //upstream positive acoustic wave
            amp_neg= 0.0;            amp_pos= amp;            endTime=perStartTime+10;            shockPos=1.5;            wavePos=-waveLength;
        } else if(config==2){ //downstream negative acoustic wave
            amp_neg= amp;            amp_pos= 0.0;            endTime=perStartTime+10;            shockPos=0.5;            wavePos=3.2;
        } else if(config==3){ //upstream positive acoustic wave
            amp_neg= amp;            amp_pos= 0.0;            endTime=perStartTime+10;            shockPos=1.5;            wavePos=-waveLength;
        }
        
        foreach(int p in DGdegrees) {
        foreach(int n in Grid_Res) {
            //int cells=(n+1)*30;
            int xCells =(n+1)*15+1;
            int yCells =3;
            var tmp_cntrl=CNS.ControlExamples_Supersonic.AcousticWave( isRestart: isRestart ,waveLength: waveLength ,savePeriod: savePeriod ,perStartTime: perStartTime ,endTime: endTime ,MachL: MachL ,wavePosition: wavePos , shockPosition: shockPos ,p_amp_neg:  amp_neg , p_amp_pos: amp_pos ,waveform:  waveform  ,dgDegree: p ,numOfCellsX: xCells ,numOfCellsY: yCells ,dbPath:dbPath  );
            Job j = new Job(tmp_cntrl.SessionName, new CNS.CNSProgram().GetType());
            // job name has to be equal to sessionname! for SessionInfoJobCorrelation to work!
                
            j.RetryCount = 2;
            j.NumberOfThreads=1;
            var tmp_restart=isRestart? "true": "false";
            j.SetControlStatement($"cs:CNS.ControlExamples_Supersonic.AcousticWave( isRestart:{tmp_restart},waveLength:{waveLength},savePeriod:{savePeriod},perStartTime:{perStartTime},endTime:{endTime},MachL:{MachL},wavePosition:{wavePos}, shockPosition:{shockPos},p_amp_neg: {amp_neg}, p_amp_pos:{amp_pos},waveform:\"{waveform}\",dgDegree:{p},numOfCellsX:{xCells},numOfCellsY:{yCells},dbPath:@\"{dbPath}\")");
            j.Activate(BoSSSshell.ExecutionQueues[0]); 
        } }                                                                                                                                                          
    } } }
} } } 

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(7*86400 , 60); // wait for one week, or until all jobs finish